# Master Kernel
##### Tiempo estimado de corrida: 9 minutos
___
### Análisis Exploratorio

##### Integrantes:
+ Alvarado, Rubén
+ Penovi, Francisco

In [ ]:
############################ROOT#############################
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

STATUS_CSV = '../input/status.csv'
STATION_CSV = '../input/station.csv'
TRIP_CSV = '../input/trip.csv'
WEATHER_CSV = '../input/weather.csv'

dfStation = pd.read_csv(STATION_CSV, parse_dates=['installation_date'], infer_datetime_format=True)
# dfStatus = pd.read_csv(STATUS_CSV)   OJO CON ESTE QUE ES MUY PESADO
dfTrip = pd.read_csv(TRIP_CSV, parse_dates=['start_date', 'end_date'], infer_datetime_format=True)
dfWeather = pd.read_csv(WEATHER_CSV, parse_dates=['date'], infer_datetime_format=True)

''' Recibe un plot de barras y les asigna sus valores
    encima de cada una de ellas. Se puede setear textsize y alpha.'''
def autolabel(ax, textsize, alpha):
    # Get y-axis height to calculate label position from.
    (y_bottom, y_top) = ax.get_ylim()
    y_height = y_top - y_bottom

    for rect in ax.patches:
        height = rect.get_height()
        label_position = height + (y_height * 0.01)

        ax.text(rect.get_x() + rect.get_width()/2., label_position,
                '%d' % int(height),
                ha='center', va='bottom', size=textsize, alpha=alpha)


''' Utilizada como parámetro para función DataFrame.apply
    Para cada row, devuelve el zip code correspondiente a la ciudad.'''
def cityNameToZipCode(row) :

    if row.city == 'San Francisco' :
        return '94107'

    if row.city == 'Redwood City' :
        return '94063'

    if row.city == 'Palo Alto' :
        return '94301'

    if row.city == 'Mountain View' :
        return '94041'

    if row.city == 'San Jose' :
        return '95113'

    raise ValueError('Malo malo!')


''' Utilizada para generar un DataFrame nuevo que contiene las filas de la ciudad
    pasada por parámetro únicamente.'''
def makeNewDataFrameWithCity(df, city) :

    dic = {'date':[], 'city':[], 'zip_code':[], 'duration':[], 'max_temp_f':[], 'min_temp_f':[]}

    for i in range(0, len(df)):
        if (df.iloc[i]['city'] == city ) :
            dic['date'].append(df.iloc[i]['date'])
            dic['city'].append(df.iloc[i]['city'])
            dic['zip_code'].append(df.iloc[i]['zip_code'])
            dic['duration'].append(df.iloc[i]['duration'])
            dic['max_temp_f'].append(df.iloc[i]['max_temperature_f'])
            dic['min_temp_f'].append(df.iloc[i]['min_temperature_f'])

    return pd.DataFrame.from_dict(data=dic, orient='columns')


''' Utilizada para clasificar en grupos a las horas del día.'''
def getMomentOfDay(row):

    if row.hour < 5:
        return 'Small Hours'
    if  5 <= row.hour < 12:
        return 'Morning'
    if 12 <= row.hour < 14:
        return 'Noon'
    if 14 <= row.hour < 18:
        return 'Afternoon'
    if 18 <= row.hour < 21:
        return 'Evening'
    if 21 <= row.hour <= 23:
        return 'Night'

###############################ENDROOT#############################

### ----- INICIO NOTEBOOK 01 -----

### Subscription Type VS Trip Duration

In [ ]:
result = dfTrip[['duration', 'start_date', 'end_date', 'subscription_type']]\
        .sort_values(by=['start_date', 'end_date'])\
        .groupby(by=['subscription_type'])\
        .agg('mean').reset_index()
        
plt.figure(figsize=(12,4))

sns.set(font_scale=1.30)
sns.set_style('ticks')

bars = sns.barplot(x='duration',\
                   y='subscription_type',\
                   order=['Subscriber', 'Customer'],\
                   data=result)

sns.despine()

bars.set(xlabel='duration (seconds)',\
         ylabel='');

bars.set_title('Average trip duration according to subscription type:', size=20, loc='left', alpha=0.85);
plt.show()

### Observo la varianza de las duraciones
Para realizar recortes de volúmenes de datos muy pequeños pero con valores muy altos o muy bajos que alteran la naturaleza de los datos.

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2)

fig.set_size_inches(14, 6)

sns.despine()

dfTrip['duration'].plot(ax=axes[0], kind='area', legend=True);
axes[0].set_title('Original');
dfTrip[(60*2 <= dfTrip.duration) & (dfTrip.duration <= 500000)][['duration']].plot(ax=axes[1], kind='area', legend=False);
axes[1].set_title('Trimmed 1');
dfTrip[(60*2 <= dfTrip.duration) & (dfTrip.duration <= 100000)][['duration']].plot(kind='area', legend=False).set_title('Trimmed 2');

sns.despine()

plt.show();

**Para facilitar la comprensión de la dispersión de los datos (ya que en los gráficos no se puede apreciar)  
voy a armar un nuevo DataFrame con la cantidad de viajes dentro de los intervalos existentes.**

In [ ]:
# Armo intervalos de duracion y me fijo cual es el intervalo más común
durationIntervals = {}
maxDuration = dfTrip.duration.max()
minDuration = dfTrip.duration.min()

while ( maxDuration >= 120) :    
    interval = (maxDuration/2 + 1, maxDuration)
    durationIntervals[interval] = 0
    maxDuration /= 2

durationIntervals[(minDuration, maxDuration)] = 0

# Cuento la cantidad de trips en cada intervalo y relleno el diccionario:
for interval in durationIntervals.keys() :
    durationIntervals[interval] = dfTrip[(interval[0] <= dfTrip.duration) & (dfTrip.duration <= interval[1])]\
                                        .duration.count()

In [ ]:
dfIntervals = pd.DataFrame.from_dict(durationIntervals, orient='index').reset_index()
dfIntervals.columns = ['interval', 'trips']
dfIntervals.sort_values(by=['interval'], inplace=True)
dfIntervals

In [ ]:
ax = dfIntervals.plot(kind='bar', x='interval', y='trips', color=sns.color_palette(), legend=False)
ax.figure.set_size_inches(15, 10)

sns.despine()
ax.set_title('Number of Trips in trip-duration intervals', size=22)
ax.set_xlabel('Intervals (seconds)', size=18, alpha=0.7)
ax.set_ylabel('Trips', size=18, rotation='horizontal', position=(0, 0.47), alpha=0.7)
plt.xticks(rotation=55, size=13)

autolabel(ax, 11.5, 0.9)      # Definida en root.py

plt.show()

### Recorto viajes menores a 2 minutos y mayores a 6 horas
Podemos observar en el gráfico de arriba que las duraciones más comunes de los viajes estan entre el rango de **4** a **17** minutos.  
Existen viajes extremadamente largos (200 días por ejemplo) que alteran los promedios de los viajes al igual que viajes extremadamente cortos que no representan casos habituales en el set de datos y perjudican a la comprensión y generalización de los mismos.

**De hecho en la página oficial el uso extendido de las bicicletas es desalentado para este servicio:**
> *"If you plan on using a bicycle for an extended period of time,
> we encourage you to rent a bike at a local bike shop or 
> rental business."*

In [ ]:
lendfTrip = len(dfTrip)

###################################################################
# Filtro dfTrip por viajes de duracion entre 2 y 6 horas

dfTrip = dfTrip[(2*60 <= dfTrip.duration) & (dfTrip.duration <= 6*3600)]

###################################################################

print ("\nAntes del recorte:", lendfTrip, "\nDespués del recorte:", len(dfTrip), "\n")
print ("Datos perdidos:", '%' + str(100 - (len(dfTrip) / float(lendfTrip))*100.0))

In [ ]:
result = dfTrip[['duration', 'start_date', 'end_date', 'subscription_type']]\
        .sort_values(by=['start_date', 'end_date'])\
        .groupby(by=['subscription_type'])\
        .agg('mean').reset_index()
        
plt.figure(figsize=(12,4))

sns.set(font_scale=1.30)
sns.set_style('ticks')

bars = sns.barplot(x='duration',\
                   y='subscription_type',\
                   order=['Subscriber', 'Customer'],\
                   data=result)

sns.despine()

bars.set(xlabel='duration (seconds)',\
         ylabel='');

bars.set_title('Average trip duration according to subscription type:', size=20, loc='left', alpha=0.85);
plt.show()

Se puede apreciar que los valores de los **suscriptos** se mantienen iguales mientras que las duraciones de los viajes de los **clientes** rondan los 45 minutos, en oposición al gráfico anterior en donde éstos últimos sobrepasaban la hora.

Esto cobra un sentido más trascendente cuando se lo observa a la par con los precios listados en la página oficial:  
[SF Bay Area Bike Share](https://www.bayareabikeshare.com/pricing "Bay Area Bike Pricing") en donde se puede observar que a partir de los 30+ minutos de viaje, comienzan a haber pequeños recargos y luego de los 60+ minutos de viaje los recargos aumentan. Por lo que tiene sentido que en promedio los viajes no se excedan de los 60 minutos.

### ----- FIN NOTEBOOK 01 -----

___

### ----- INICIO NOTEBOOK 02 -----

### Temperature VS Trip Duration
**Analizando el DataFrame de Weather:**

In [ ]:
dfWeather.isnull().sum()

In [ ]:
dfWeather[dfWeather.date == pd.to_datetime('2013-08-29', format='%Y-%m-%d')][['date', 'zip_code']]

**Observamos que para cada fecha tenemos pronósticos de 5 zipcodes distintos:**  
Investigando, los zipcodes se corresponden de la siguiente manera

+ 94107 --> San Francisco
+ 94063 --> Redwood City
+ 94301 --> Palo Alto
+ 94041 --> Mountain View
+ 95113 --> San Jose

### Analizando el Dataframe de Trips
**Mientras que los zipcodes de la tabla Trips son de los usuarios que usan las bicis:**

In [ ]:
print (dfTrip.zip_code.unique())
print ("Zip Codes distintos:", dfTrip.zip_code.unique().size)

Por ejemplo:

+ 94109   --> San Francisco
+ 95112   --> San Jose
+ 94041   --> Mountain View
+ 2280    --> Bélgica
+ 9240    --> Bélgica
+ 29609   --> Greenville

*Estos zip codes en principio no nos servirían para nada más que para saber de donde es la persona que realizó el viaje*

**Las ciudades en las que se encuentran las estaciones son las mismas de las que tenemos información meteorológica:**  
Esto implica que podremos vincular el dato de la duración de un viaje en particular por su start_station_id con una estación en la tabla de stations y así con una ciudad y su información meteorológica.

In [ ]:
dfStation.city.unique()

### Ploteando :
Voy a necesitar una tabla con estos datos:

| Trip_date     | Duration      | min_temp  | max_temp
|:-------------:|:-------------:|:---------:|:--------:
| XXXX-XX-XX    | xxx           | xx °F     | xx °F

Donde Duration va a ser la media/mediana o el promedio de los viajes en ese dia.
Podria hacer un multiplot para los 5 zipcodes

In [ ]:
dfTrip_N_Weather = dfTrip[['duration', 'start_date', 'start_station_id']]
dfTrip_N_Weather.rename(columns={'start_station_id':'id'}, inplace=True)
dfTrip_N_Weather = dfTrip_N_Weather.merge(dfStation[['id', 'city']], on=['id'])

In [ ]:
dfTrip_N_Weather.sort_values(by='duration')
dfTrip_N_Weather['zip_code'] = dfTrip_N_Weather.apply(cityNameToZipCode, axis=1)
dfTrip_N_Weather.head()

In [ ]:
dfTrip_N_Weather.start_date = pd.to_datetime(dfTrip_N_Weather.start_date.dt.date)

In [ ]:
dfTrip_N_Weather.rename(columns={'start_date':'date'}, inplace=True)
dfTrip_N_Weather = dfTrip_N_Weather[['duration', 'date', 'city', 'zip_code']].groupby(by=['date', 'city', 'zip_code'])\
                                                          .agg('mean')\
                                                          .reset_index()

# Como el zip code tiene type int64, lo paso a str para poder hacer el merge
# dfTrip_N_Weather.zip_code = int(dfTrip_N_Weather.zip_code)

In [ ]:
# Convierto a int64 para poder hacer el merge
dfTrip_N_Weather.zip_code = pd.to_numeric(dfTrip_N_Weather.zip_code)

In [ ]:
dfTrip_N_Weather = dfTrip_N_Weather.merge(dfWeather[['date', 'zip_code', 'max_temperature_f', 'min_temperature_f']], on=['date', 'zip_code'])

In [ ]:
dfTrip_N_Weather.sort_values(by='date', inplace=True)
dfTrip_N_Weather.head()

### Análisis mediante Visualización:
Primero haremos una visualización de **todas las ciudades en conjunto** de forma genérica para ver si se puede observar un patrón. Luego quizás puede ser interesante ver el comportamiento de los viajes en cada ciudad en particular.

In [ ]:
dfTrip_N_WeatherGeneral = dfTrip_N_Weather.sort_values(by=['date'])
dfTrip_N_WeatherGeneral[['date', 'duration', 'max_temperature_f', 'min_temperature_f']]\
                        .groupby(by=['date'])\
                        .agg('mean')\
                        .reset_index()\
                        .plot(x='date',\
                              kind='line',\
                              secondary_y='duration',\
                              figsize=(16,14),\
                              color=['g', '#cc5245', '#10b9f2']
                              )
plt.show()

### Plot General fecha a fecha ↑ :
**En principio este gráfico es muy confuso, por la cantidad de registros que hay.**  
Lo que podemos hacer a continuación es, realizar el mismo grafico pero con los promedios de duraciones mes a mes, de esta forma obtendremos un gráfico más sobrio.

In [ ]:
dfTrip_N_WeatherGeneral.date = dfTrip_N_WeatherGeneral.date.dt.month
ax = dfTrip_N_WeatherGeneral[['date', 'duration', 'max_temperature_f', 'min_temperature_f']]\
                    .groupby(by=['date'])\
                    .agg('mean')\
                    .reset_index()\
                    .plot(kind='line',\
                          x='date',\
                          secondary_y='duration',\
                          figsize=(14,10),\
                          color=['g', '#cc5245', '#10b9f2'])

for line in ax.get_lines() :
    line.set_linewidth(5)
    line.set_antialiased(True)
    line.set_alpha(0.5)

ax.legend(fontsize=12, bbox_to_anchor=(1, 0.968))
ax.right_ax.legend(fontsize=13)
ax.right_ax.get_lines()[0].set_linewidth(3)
ax.right_ax.set_ylabel("Duration (seconds)", fontsize=18, alpha=0.7)

for yticklabel in ax.right_ax.get_yticklabels() :
    yticklabel.set_size(12)

ax.set_xticks(range(1,13))
ax.set_xticklabels(['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'], fontsize=14)
ax.set_xlabel("Months", fontsize=18, alpha=0.7)
ax.set_ylabel("Temp (F)", fontsize=18, alpha=0.7)

for xticklabel in ax.get_yticklabels() :
    xticklabel.set_size(12)
    
ax.set_facecolor('#afbacc')
ax.patch.set_alpha(0.5)
plt.title('Average trip duration VS temperature', fontsize=24)
plt.show()

### Plot General mes a mes ↑ :
Plot de mes a mes los promedios de duraciones de todos los viajes (5 ciudades) tomando en cuenta los 3 años.
#### Observaciones:
+ Podemos resaltar que los viajes de mayor duración ocurrieron en Julio, época de mayor calor.
+ Además Julio-Agosto es la época en donde California se encuentra más llena de turistas quienes son los que         realizan los viajes de mayores duraciones ya que la mayoría de la gente que usa las bicis como medio de             transporte hacia su trabajo no superan los 30 minutos de viaje.
+ La caída en septiembre y octubre puede deberse a que comienzan las clases en esa época, por lo que los alumnos     quizas no utilizan las bicicletas durante el período lectivo.

### Armo 5 DataFrames para poder plotear cada ciudad por separado

In [ ]:
dfRedwood = makeNewDataFrameWithCity(dfTrip_N_Weather, 'Redwood City')
dfSanFrancisco = makeNewDataFrameWithCity(dfTrip_N_Weather, 'San Francisco')
dfSanJose = makeNewDataFrameWithCity(dfTrip_N_Weather, 'San Jose')
dfMountainView = makeNewDataFrameWithCity(dfTrip_N_Weather, 'Mountain View')
dfPaloAlto = makeNewDataFrameWithCity(dfTrip_N_Weather, 'Palo Alto')
print (len(dfRedwood), len(dfSanFrancisco), len(dfSanJose), len(dfMountainView), len(dfPaloAlto))

In [ ]:
dfRedwood.date = dfRedwood.date.dt.month
dfRedwood = dfRedwood[['date', 'duration', 'max_temp_f', 'min_temp_f']]
ax = dfRedwood.groupby(by=['date'])\
              .agg('mean')\
              .reset_index()\
              .plot(kind='line',\
                    x='date',\
                    secondary_y='duration',\
                    figsize=(14,10),\
                    color=['g', '#cc5245', '#10b9f2'])
            
for line in ax.get_lines() :
    line.set_linewidth(5)
    line.set_antialiased(True)
    line.set_alpha(0.5)

ax.legend(fontsize=12, bbox_to_anchor=(1, 0.968))
ax.right_ax.legend(fontsize=13)
ax.right_ax.get_lines()[0].set_linewidth(3)
ax.right_ax.set_ylabel("Duration (seconds)", fontsize=18, alpha=0.7)

for yticklabel in ax.right_ax.get_yticklabels() :
    yticklabel.set_size(12)

ax.set_xticks(range(1,13))
ax.set_xticklabels(['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'], fontsize=14)
ax.set_xlabel("Months", fontsize=18, alpha=0.7)
ax.set_ylabel("Temp (F)", fontsize=18, alpha=0.7)

for xticklabel in ax.get_yticklabels() :
    xticklabel.set_size(12)
    
ax.set_facecolor('#afbacc')
ax.patch.set_alpha(0.5)

plt.title('Redwood City: Average trip duration VS temperature', fontsize=24)
plt.show()

### Reedwood City ↑ :
Pareciera estar coincidiendo con el esquema general anterior en las siguientes componentes:

+ Pico de duraciones en Agosto.
+ Decremento drástico en Septiembre.

In [ ]:
dfSanFrancisco.date = dfSanFrancisco.date.dt.month
dfSanFrancisco = dfSanFrancisco[['date', 'duration', 'max_temp_f', 'min_temp_f']]
ax = dfSanFrancisco.groupby(by=['date'])\
              .agg('mean')\
              .reset_index()\
              .plot(kind='line',\
                    x='date',\
                    secondary_y='duration',\
                    figsize=(14,10),\
                    color=['g', '#cc5245', '#10b9f2'])
            
for line in ax.get_lines() :
    line.set_linewidth(5)
    line.set_antialiased(True)
    line.set_alpha(0.5)

ax.legend(fontsize=12, bbox_to_anchor=(1, 0.968))
ax.right_ax.legend(fontsize=13)
ax.right_ax.get_lines()[0].set_linewidth(3)
ax.right_ax.set_ylabel("Duration (seconds)", fontsize=18, alpha=0.7)

for yticklabel in ax.right_ax.get_yticklabels() :
    yticklabel.set_size(12)

ax.set_xticks(range(1,13))
ax.set_xticklabels(['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'], fontsize=14)
ax.set_xlabel("Months", fontsize=18, alpha=0.7)
ax.set_ylabel("Temp (F)", fontsize=18, alpha=0.7)

for xticklabel in ax.get_yticklabels() :
    xticklabel.set_size(12)
    
ax.set_facecolor('#afbacc')
ax.patch.set_alpha(0.5)

plt.title('San Francisco: Average trip duration VS temperature', fontsize=24)
plt.show()

### San Francisco ↑ :
San Francisco en cambio coincide con la suba en Agosto pero su pico máximo se ve en Septiembre.

In [ ]:
dfSanJose.date = dfSanJose.date.dt.month
dfSanJose = dfSanJose[['date', 'duration', 'max_temp_f', 'min_temp_f']]
ax = dfSanJose.groupby(by=['date'])\
              .agg('mean')\
              .reset_index()\
              .plot(kind='line',\
                    x='date',\
                    secondary_y='duration',\
                    figsize=(14,10),\
                    color=['g', '#cc5245', '#10b9f2'])
            
for line in ax.get_lines() :
    line.set_linewidth(5)
    line.set_antialiased(True)
    line.set_alpha(0.5)

ax.legend(fontsize=12, bbox_to_anchor=(1, 0.968))
ax.right_ax.legend(fontsize=13)
ax.right_ax.get_lines()[0].set_linewidth(3)
ax.right_ax.set_ylabel("Duration (seconds)", fontsize=18, alpha=0.7)

for yticklabel in ax.right_ax.get_yticklabels() :
    yticklabel.set_size(12)

ax.set_xticks(range(1,13))
ax.set_xticklabels(['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'], fontsize=14)
ax.set_xlabel("Months", fontsize=18, alpha=0.7)
ax.set_ylabel("Temp (F)", fontsize=18, alpha=0.7)

for xticklabel in ax.get_yticklabels() :
    xticklabel.set_size(12)
    
ax.set_facecolor('#afbacc')
ax.patch.set_alpha(0.5)

plt.title('San Jose: Average trip duration VS temperature', fontsize=24)
plt.show()

### San Jose ↑ :
Otra vez vemos que coincide que los viajes de mayor duración se ven en Agosto-Septiembre en la época de mayor temperatura, esto ya no es coincidencia.

In [ ]:
dfMountainView.date = dfMountainView.date.dt.month
dfMountainView = dfMountainView[['date', 'duration', 'max_temp_f', 'min_temp_f']]
ax = dfMountainView.groupby(by=['date'])\
              .agg('mean')\
              .reset_index()\
              .plot(kind='line',\
                    x='date',\
                    secondary_y='duration',\
                    figsize=(14,10),\
                    color=['g', '#cc5245', '#10b9f2'])
            
for line in ax.get_lines() :
    line.set_linewidth(5)
    line.set_antialiased(True)
    line.set_alpha(0.5)

ax.legend(fontsize=12, bbox_to_anchor=(1, 0.968))
ax.right_ax.legend(fontsize=13)
ax.right_ax.get_lines()[0].set_linewidth(3)
ax.right_ax.set_ylabel("Duration (seconds)", fontsize=18, alpha=0.7)

for yticklabel in ax.right_ax.get_yticklabels() :
    yticklabel.set_size(12)

ax.set_xticks(range(1,13))
ax.set_xticklabels(['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'], fontsize=14)
ax.set_xlabel("Months", fontsize=18, alpha=0.7)
ax.set_ylabel("Temp (F)", fontsize=18, alpha=0.7)

for xticklabel in ax.get_yticklabels() :
    xticklabel.set_size(12)
    
ax.set_facecolor('#afbacc')
ax.patch.set_alpha(0.5)

plt.title('Mountain View: Average trip duration VS temperature', fontsize=24)
plt.show()

### Mountain View ↑ :
Reafirmando lo visto por el gráfico general y los casos particulares de las ciudades anteriores, definitivamente los viajes con mayores duraciones se dan a temperaturas mayores.

In [ ]:
dfPaloAlto.date = dfPaloAlto.date.dt.month
dfPaloAlto = dfPaloAlto[['date', 'duration', 'max_temp_f', 'min_temp_f']]
ax = dfPaloAlto.groupby(by=['date'])\
              .agg('mean')\
              .reset_index()\
              .plot(kind='line',\
                    x='date',\
                    secondary_y='duration',\
                    figsize=(14,10),\
                    color=['g', '#cc5245', '#10b9f2'])
            
for line in ax.get_lines() :
    line.set_linewidth(5)
    line.set_antialiased(True)
    line.set_alpha(0.5)

ax.legend(fontsize=12, bbox_to_anchor=(1, 0.968))
ax.right_ax.legend(fontsize=13)
ax.right_ax.get_lines()[0].set_linewidth(3)
ax.right_ax.set_ylabel("Duration (seconds)", fontsize=18, alpha=0.7)

for yticklabel in ax.right_ax.get_yticklabels() :
    yticklabel.set_size(12)

ax.set_xticks(range(1,13))
ax.set_xticklabels(['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'], fontsize=14)
ax.set_xlabel("Months", fontsize=18, alpha=0.7)
ax.set_ylabel("Temp (F)", fontsize=18, alpha=0.7)

for xticklabel in ax.get_yticklabels() :
    xticklabel.set_size(12)
    
ax.set_facecolor('#afbacc')
ax.patch.set_alpha(0.5)

plt.title('Palo Alto: Average trip duration VS temperature', fontsize=24)
plt.show()

### Palo Alto ↑ :
Quizas el caso más conflictivo es en esta ciudad, donde el comportamiento de las duraciones de los viajes es bastante errático. Mantiene lo ya identificado: **la mayor duración de los viajes en la época más calurosa**, sin embargo, en diciembre y enero presenta viajes bastante largos con temperaturas bajas.

### ----- FIN NOTEBOOK 02 -----

___

### ----- INICIO NOTEBOOK 03 -----

In [ ]:
dfTripReduced = dfTrip[['start_date', 'duration']].sort_values(by=['start_date'])
#Se realiza conversión de segundos a minutos en duración para mayor claridad en los gráficos
dfTripReduced['duration'] = dfTripReduced['duration'].apply(lambda seconds: float(seconds)/60)


dfTripReduced['hour'] = dfTripReduced.apply(lambda row: row.start_date.hour, axis=1)

dfTripsByHour = dfTripReduced[['hour', 'duration']].groupby(by=['hour'])\
                    .agg(['mean', 'count'])['duration']\
                    .rename(columns={'mean': 'avg_duration', 'count':'trips'})

sns.set_style("ticks")

ax = dfTripsByHour.plot(kind='bar', y='trips', color=sns.color_palette(), legend=False)

ax.figure.set_size_inches(15, 10)
ax.set_title('Average amount of Trips throughout a day', size=22)
ax.set_xlabel('Day Hours', size=18, alpha=0.7)
ax.set_ylabel('N of Trips', size=18, rotation='horizontal', position=(0, 0.47), alpha=0.7)
plt.xticks(rotation=55, size=13)
autolabel(ax, 11, 0.8)
sns.despine()
plt.show()

## Calculo de promedio de duración de los viajes de acuerdo a momento del día

##### Debido a que la cantidad de viajes durante la madrugada (entre las 0 hs y las 4 hs) es menor, es probable  que los promedios de duración sean muy altos o bajos de acuerdo al caso por lo que no seran relevantes en el análisis 

In [ ]:
dfTripReduced['moment_of_day'] =  dfTripReduced.apply(getMomentOfDay, axis=1)

dfMomentOfDay = dfTripReduced[['moment_of_day', 'duration']]\
                                .groupby(by=['moment_of_day'])\
                                .agg('mean').reset_index()        

sns.set(font_scale=1.30)
sns.set_style('ticks')

bars = sns.barplot(y='duration',\
                   x='moment_of_day',\
                   order=['Small Hours', 'Morning', 'Noon', 'Afternoon', 'Evening', 'Night'],\
                   data=dfMomentOfDay)

sns.despine()

bars.set(ylabel='duration (minutes)',\
         xlabel='\nMoment of the day');

bars.set_xticklabels(['Small Hours\n[0 - 4]', 'Morning\n[5 - 11]', \
                      'Noon\n[12 - 13]', 'Afternoon\n[14 - 17]', \
                      'Evening\n[18 - 20]', 'Night\n[21 - 23]'])
bars.set_yticklabels

bars.set_title('Average trip duration according to moment of the day\n', size=20, loc='left', alpha=0.85);
autolabel(bars, 11.5, 0.9)
plt.show()

In [ ]:
sns.set_style('ticks')

ax = dfTripReduced[['hour', 'duration']]\
            .groupby(by=['hour'])\
            .agg('mean')\
            .reset_index()\
            .plot(kind='bar',
                  x='hour',
                  y='duration',
                  figsize=(16,10), 
                  color=sns.color_palette("coolwarm", 24), 
                  legend=False,
                  rot=0)
            
sns.despine()
ax.set_title('Hour-To-Hour Average durations', fontsize=24)
ax.set_xlabel("\nDay Hours", fontsize=16, alpha=0.8)
ax.set_ylabel("Duration (minutes)", fontsize=16, alpha=0.8)
ax.set_facecolor('#916a96')
ax.patch.set_alpha(0.6)
autolabel(ax, 13, 1)
plt.show()

In [ ]:
#Hago merge para tener los tres ejes: hora, duracion y cantidad de viajes por hora
dfMerged = dfTripReduced.merge(dfTripsByHour.reset_index(), on='hour')
dfMerged['floatHour'] = dfMerged.apply(lambda x: x.start_date.hour + float(x.start_date.minute)/60, axis=1)

sns.set_style("ticks", rc={'legend.scatterpoints': 5, 'legend.numpoints': 5})

graf = sns.lmplot(x='floatHour', y='duration',
           data=dfMerged,
           fit_reg=False,
           hue="trips",
           palette="YlOrRd",
           scatter_kws={"s":12},
           legend_out=False,
          )

plt.legend(loc='upper left', fontsize=12, title='Number Of Trips\n')
graf.fig.set_size_inches(15, 15)
graf.ax.patch.set_facecolor('#d6d6d6')
graf.ax.patch.set_alpha(0.7)
plt.title('Histogram of trips\n', fontsize=26, loc='left')
plt.xlabel('Day hours', fontsize=18, alpha=0.8)
plt.ylabel('Duration (minutes)', fontsize=18, alpha=0.8)
#En el eje X se representan las 24 horas del día y en Y rangos de a 60 minutos (1 hora)
graf.set(xlim=(0, 24), xticks=[x for x in range(25) ], yticks=[i * 60 for i in range(7)])
plt.show()

### ----- FIN NOTEBOOK 03 -----

___

### ----- INICIO NOTEBOOK 04 -----

## Precipitations VS Duration

#### En principio se buscara visualizar la distribucion de la duración de los viajes de acuerdo a las diferentes medidas de precipitaciones

In [ ]:
#Descarto las rows que no me brindan datos acerca de las precipitaciones
dfPrecipitations = dfWeather[['date', 'zip_code', 'precipitation_inches', 'events']]
dfPrecipitations.precipitation_inches = dfPrecipitations.precipitation_inches.convert_objects(convert_numeric=True)
#Los no numericos quedan como NaN y los limpio
dfPrecipitations = dfPrecipitations[dfPrecipitations.precipitation_inches.notnull()]
print ('Se limpiaron %d registros' % (len(dfWeather) - len(dfPrecipitations)))

In [ ]:
dfPrecipitations.columns

In [ ]:
dfTrip_N_Precipitations = dfTrip[['duration', 'start_date', 'start_station_id']]
dfTrip_N_Precipitations.rename(columns={'start_station_id':'id'}, inplace=True)
dfTrip_N_Precipitations = dfTrip_N_Precipitations.merge(dfStation[['id', 'city']], on=['id'])

dfTrip_N_Precipitations.sort_values(by='duration')

################### Realizo la conversión para hacer merge de los datos ###############################
dfTrip_N_Precipitations['zip_code'] = dfTrip_N_Precipitations.apply(cityNameToZipCode, axis=1)
dfTrip_N_Precipitations.start_date = pd.to_datetime(dfTrip_N_Precipitations.start_date.dt.date)

dfTrip_N_Precipitations.rename(columns={'start_date':'date'}, inplace=True)
dfTrip_N_Precipitations = dfTrip_N_Precipitations[['duration', 'date', 'city', 'zip_code']]
# Convierto a int64 para poder hacer el merge
dfTrip_N_Precipitations.zip_code = pd.to_numeric(dfTrip_N_Precipitations.zip_code)
dfTrip_N_Precipitations = dfTrip_N_Precipitations.merge(dfPrecipitations, on=['date', 'zip_code'])

dfTrip_N_Precipitations.sort_values(by='date', inplace=True)
dfTrip_N_Precipitations.head()

In [ ]:
sns.set(font_scale=1.30)
sns.set_style("ticks")

graf = sns.lmplot(x='precipitation_inches', y='duration',
           data=dfTrip_N_Precipitations,
           fit_reg=False,
           scatter_kws={"s":10}
          )
graf.fig.set_size_inches(16, 9)
           
plt.title('General Trip avg duration distribution against precipitation')
plt.xlabel('Precipitation inches', fontsize=16, alpha=0.8)
plt.ylabel('Duration (seconds)', fontsize=16, alpha=0.8)
plt.show()

#### Se puede observar que a medida que las precipitaciones son mas copiosas, los viajes comienzan a descender en duracion y también en cantidad.
#### Por otra parte se puede ver que los viajes iniciados en Mountain View son en su grán mayoría en dias en donde las precipitaciones estan en 0. Si bien esta tendencia se puede ver en todas las ciudades, la diferencia en Mountain View es mucho mas marcada.
#### Algo raro que también se puede observar es que aparecen algunos viajes a partir de los 3 inches.

In [ ]:
print ('Viajes en los que el nivel de precipitaciones es menor a 2 inches es: %d' % len(dfTrip_N_Precipitations[dfTrip_N_Precipitations.precipitation_inches < 2]))
print ('Viajes en los que el nivel de precipitaciones es mayor a 3 inches es: %d' % len(dfTrip_N_Precipitations[dfTrip_N_Precipitations.precipitation_inches > 3]))

#### Si bien la cantidad de viajes, cuando las precipitaciones son de mas de 3 inches, es muy baja, se verifica ver por qué razón podría la gente tomar bicis en este nivel y ninguna cuando es de 1 a 2 (En este caso la cantidad es 0)

In [ ]:
dfTrip_N_Precipitations[dfTrip_N_Precipitations.precipitation_inches > 3].events.unique()

#### Se puede ver que los registros en los que el valor 'Precipitation inches' es mayor a 3 habia lluvia, al parecer muy copiosa, pero no tormenta. Visto esto se procede a realizar el mismo gráfico anterior tomando en cuenta los eventos meteorológicos.

In [ ]:
dfTrip_N_PrecipNormalizedEvents = dfTrip_N_Precipitations[dfTrip_N_Precipitations.events.notnull()]
dfTrip_N_PrecipNormalizedEvents.events = dfTrip_N_PrecipNormalizedEvents.events.apply(lambda event: event.capitalize())
dfTrip_N_PrecipNormalizedEvents.head()

In [ ]:
sns.set(font_scale=1.30)
sns.set_style("ticks")
flatui = ["#9b59b6", "#3498db","#e74c3c", "#34495e", "#2ecc71"]
graf = sns.lmplot(x='precipitation_inches', y='duration',
           data=dfTrip_N_PrecipNormalizedEvents,
           fit_reg=False,
           hue = "events",
           palette = flatui,
           scatter_kws={"s":12},
           legend_out=False
          )
graf.fig.set_size_inches(16, 9)
plt.legend(loc='upper right', fontsize=12, title='Weather Events')
plt.title('Trip avg duration distribution in precipitation inches', fontsize=20)
plt.xlabel('Precipitation inches', fontsize=16, alpha=0.8)
plt.ylabel('Duration (seconds)', fontsize=16, alpha=0.8)
plt.show()

#### Se puede observar que la gente solo alquila en dias de tormenta eléctrica cuando el nivel de precipitación es muy cercano a 0.
#### Se realiza un plot un poco mas adecuado para mostrar esto ya que la duración no nos está dando muchos datos relevantes.

In [ ]:
dfTrip_N_PrecipNormalizedEvents.precipitation_inches = dfTrip_N_PrecipNormalizedEvents.precipitation_inches.apply(lambda preInch: int(preInch))
sns.set(style="ticks")
countPlot = sns.countplot(x="precipitation_inches", hue="events", data=dfTrip_N_PrecipNormalizedEvents, palette = flatui)
sns.despine()
countPlot.figure.set_size_inches(18, 8)
countPlot.set_xticklabels(['[0 - 1]', '[1 - 2]', '[3 - 4]'])
countPlot.legend(loc='right')

plt.show()

### ----- FIN NOTEBOOK 04 -----

___

### ----- INICIO NOTEBOOK 09 -----

### Days of the week VS Trip Duration

In [ ]:
dfDaysOfWeekTrips = dfTrip[['duration', 'start_date']]
daysOfWeek = ['Monday', 'Tuesday', 'Wednesday', 'Thursday\n', 'Friday', 'Saturday', 'Sunday']

In [ ]:
dfDaysOfWeekTrips.rename(columns={'start_date':'week_day'}, inplace=True)

In [ ]:
dfDaysOfWeekTrips['day_index'] = dfDaysOfWeekTrips.week_day.dt.dayofweek
dfDaysOfWeekTrips.week_day = dfDaysOfWeekTrips.week_day.dt.weekday_name
dfDaysOfWeekTrips.head()

In [ ]:
sns.set_style('ticks')
ax = dfDaysOfWeekTrips[['duration', 'day_index']] \
                      .groupby(by=['day_index']) \
                      .agg(['mean', 'count'])['duration']\
                      .rename(columns={'mean':'avg_duration', 'count':'trips'})\
                      .reset_index()\
                      .sort_values(by=['day_index'])\
                      .plot(kind='bar', \
                            x='day_index', \
                            y='avg_duration', \
                            figsize=(16, 10), \
                            legend=False, \
                            rot=0, \
                            color=['#FF8C00', '#4bb0b1', '#E9967A', '#f8a0c8', '#1db87c', '#754998', '#FF4500'])

ax.set_xlabel("Week Days", fontsize=18, alpha=0.8)
ax.set_ylabel("Duration (seconds)", fontsize=18, alpha=0.8)
ax.set_title("Week's average Trip Duration", fontsize=24)
ax.set_xticklabels(daysOfWeek, fontsize=14)
sns.despine()
autolabel(ax, 14, 0.8)
plt.show()

In [ ]:
dfGroupedAgg = dfDaysOfWeekTrips[['duration', 'day_index']] \
                                .groupby(by=['day_index']) \
                                .agg(['mean', 'count'])['duration'] \
                                .rename(columns={'mean':'avg_duration', 'count':'trips'}) \
                                .reset_index()
dfGroupedAgg

In [ ]:
plt.figure(figsize=(16,10));
sns.set_style('ticks')
ax = sns.barplot(data=dfGroupedAgg, \
                 x='day_index', \
                 y='avg_duration', \
                 hue='trips', \
                 palette=['#FF4500', '#754998', '#E9967A', '#f8a0c8', '#1db87c', '#4bb0b1', '#FF8C00'])
                 #palette=sns.color_palette("Reds_d", n_colors=7, desat=1))

ax.set_xlabel("Week Days", fontsize=18, alpha=0.8)
ax.set_ylabel("Duration (seconds)", fontsize=18, alpha=0.8)
ax.set_title("Week's average Trip Duration", fontsize=24)
ax.set_xticklabels(daysOfWeek, fontsize=16)
ax.legend(fontsize=15, loc = "upper left", title = "Number of Trips")
sns.despine()
plt.show()

### Apreciaciones:
+ Se puede observar **claramente** como durante los días de semana la cantidad se mantiene estable en su duración,   esto nos dice mucho acerca del uso de las bicicletas en la semana como medio de transporte para ir a trabajar.
+ Además los días de semana hay **mayor cantidad de viajes que los fines de semana**.
+ Los viernes se observa un leve incremento en las duraciones.
+ En los fines de semana como era de esperar se duplican las duraciones de los viajes, sin embargo no hay que         olvidarse de revisar la cantidad de muestras en estos días para no caer como víctimas de un desafortunado caso de   alta variabilidad por falta de registros.  
  En éste caso, nosotros opinamos que estamos a salvo observando que son solamente 3 veces menores en cantidad que   en el resto de los días.

### ----- FIN NOTEBOOK 09 -----